#### Prerequesites:
* Define schema (make figure) 
* Define structure (make figure)
* Define schema class in python (src/models/model_schemata.py)
* Convert schema + structure into model template (models/templates/p_model.pl)

#### Procedure:
* Load data
* Adjust model script for lfi accordingly
* Create evidence file (if I don't need different evidence files, move to builmodel notebook)
* LFI

Multiple instances, 1 pst, X enzymes, one sample per instance, multiple enzymes per instance, test how many enzymes can have in one instances, how computing time depends on N p::f for 60 samples

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import random

from src.models import model_schemata as schema
from src.models import build_model as build
from src.visualization import visualize as viz

In [3]:
os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
# os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

#### Training/Testing

In [4]:
# reading csv from files into dict
data = {}
data['e_ksea'] = pd.read_csv('data/processed/ebdt_data/run3/e_ksea.csv')
data['p_fc'] = pd.read_csv('data/processed/ebdt_data/run3/p_fc.csv')
data['es_interaction'] = pd.read_csv('data/processed/ebdt_data/run3/es_interaction_top7enz.csv')
data['e_ksea']

,enzyme,sample,tc,value,prob,p_dec,p_inc
0,ABL1,AC220,10.0,inc,0.959348,0.001000,0.959348
1,ABL1,AT13148,10.0,inc,0.627292,0.001000,0.627292
2,ABL1,AZ20,10.0,inc,0.591391,0.001000,0.591391
3,ABL1,AZD1480,10.0,dec,0.568719,0.568719,0.001000
4,ABL1,AZD3759,10.0,inc,0.534847,0.001000,0.534847
...,...,...,...,...,...,...,...
16770,WNK2,Torin,1.0,dec,0.568299,0.568299,0.001000
16771,WNK2,Trametinib,1.0,dec,0.737145,0.737145,0.001000
16772,WNK2,U73122,1.0,inc,0.747666,0.001000,0.747666
16773,WNK2,Ulixertinib,1.0,inc,0.749500,0.001000,0.749500


Splitting data into training/testing

In [5]:
# getting sample names (union of all samples in all datasets)
samples = list(set(data['e_ksea']['sample']).union(set(data['p_fc']['sample'])))
samples.sort()
# sample x% of samples randomly without replacement with seed
random.seed(612)
train = random.sample(samples, int(len(samples)*1))
print(train)
# filter data
training_data = {}
training_data['e_ksea'] = data['e_ksea'][data['e_ksea']['sample'].isin(train)].reset_index(drop=True)
training_data['p_fc'] = data['p_fc'][data['p_fc']['sample'].isin(train)].reset_index(drop=True)

['TAK715', 'TGX221', 'CHIR99021', 'CAL101', 'Ripasudil', 'KN93', 'LY2584702', 'LY2090314', 'Ku0063794', 'SP600125', 'Ribociclib', 'U73122', 'AZD5438', 'GDC0994', 'Edelfosine', 'PF3758309', 'Amuvatinib', 'AT13148', 'AZD3759', 'Go6976', 'JNJ', 'NU7441', 'GSK2334470', 'Ipatasertib', 'AZD5363', 'LY2835219', 'Torin', 'Bosutinib', 'H89', 'DNAPK', 'GDC0941', 'FRAX486', 'CX4945', 'HS173', 'Linsitinib', 'Dabrafenib', 'AZD1480', 'BX912', 'AZ20', 'TBCA', 'Selumetinib', 'GF109203X', 'AZD6482', 'AC220', 'PH797804', 'AZD8055', 'Trametinib', 'JNK', 'GSK690693', 'KD025', 'Vemurafenib', 'PF4708671', 'Tofacitinib', 'MK2206', 'GO6983', 'Dasatinib', 'AZD6738', 'PIK294', 'PD153035', 'KN62', 'Ulixertinib']


In [6]:
import multiprocessing
import concurrent.futures

# Define a function to process a single sample
def process_sample(sample, phosphosites, interactions, predicates, evidence_dict):
    evidence_dict[sample] = {}
    count = -1

    # Precompute data
    evid_generator_e = build.ProblogStatementGenerator(predicates['e_ksea'])
    evid_generator_p = build.ProblogStatementGenerator(predicates['p_fc'])

    for p in phosphosites:
        count += 1
        # Print count if divisible by 1000 to track progress
        if count % 100 == 0:
            print(f"{sample}: {count}")

        enzymes = interactions['enzyme'][interactions['substrate'] == p].tolist()

        evidence_e = []
        for e in enzymes:
            evidence_e.extend(evid_generator_e.generate_facts(build.EvidenceTemplate, select=[sample, e]))

        evidence_p = evid_generator_p.generate_facts(build.EvidenceTemplate, select=[sample, p])
        evidence_dict[sample][p] = evidence_e + evidence_p

    return evidence_dict

if __name__ == "__main__":
    num_processes = multiprocessing.cpu_count()  # Use all available CPU cores

    samples = list(set(training_data['e_ksea']['sample']) | set(training_data['p_fc']['sample']))
    phosphosites = list(set(training_data['p_fc']['phosphosite']))
    evidence_dict = {}

    # Create a ThreadPoolExecutor with the desired number of threads
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_processes) as executor:
        futures = []

        for sample in samples:
            # Precompute data
            # Mapping data to Problog predicates
            predicates = {}
            predicates['e_ksea'] = schema.EKseaPredicate()
            predicates['e_ksea'].add_data(training_data['e_ksea'][training_data['e_ksea']['sample'] == sample], 'enzyme', 'sample', 'value')
            predicates['p_fc'] = schema.PFoldChangePredicate()
            predicates['p_fc'].add_data(training_data['p_fc'][training_data['p_fc']['sample'] == sample], 'phosphosite', 'sample', 'value')
            future = executor.submit(process_sample, sample, phosphosites, data['es_interaction'], predicates, evidence_dict)
            futures.append(future)

        # Wait for all tasks to complete and retrieve their results
        for future in concurrent.futures.as_completed(futures):
            evidence_dict = future.result()

    # Now, evidence_dict contains the results of parallel processing using ThreadPoolExecutor

Dasatinib: 0
FRAX486: 0
AZD5438: 0
CX4945: 0
AZD3759: 0
AZD1480: 0
CX4945: 100
GDC0994: 0
Dasatinib: 100
PH797804: 0
AZD5438: 100
FRAX486: 100
AZD1480: 100
Dasatinib: 200
AZD3759: 100
CX4945: 200
AZD5438: 200
PH797804: 100
GDC0994: 100
FRAX486: 200AZD1480: 200

AZD3759: 200
PH797804: 200
Dasatinib: 300
AZD5438: 300
CX4945: 300
GDC0994: 200
AZD3759: 300
AZD1480: 300
Dasatinib: 400
CX4945: 400
FRAX486: 300
AZD5438: 400
PH797804: 300
AZD3759: 400
GDC0994: 300
AZD1480: 400
FRAX486: 400
CX4945: 500
AZD5438: 500
Dasatinib: 500
GDC0994: 400
AZD3759: 500
PH797804: 400
FRAX486: 500
AZD1480: 500
Dasatinib: 600
GDC0994: 500
AZD5438: 600
CX4945: 600
AZD3759: 600PH797804: 500

AZD1480: 600
FRAX486: 600
Dasatinib: 700
AZD3759: 700
FRAX486: 700
PH797804: 600
CX4945: 700
GDC0994: 600
AZD1480: 700AZD5438: 700

Dasatinib: 800
FRAX486: 800
GDC0994: 700
AZD3759: 800
PH797804: 700
AZD5438: 800
CX4945: 800
AZD1480: 800
Dasatinib: 900
FRAX486: 900GDC0994: 800

AZD3759: 900
AZD5438: 900
PH797804: 800
CX4945: 

In [7]:
# write evidence_dict to text file: loop over samples and separate by line of hyphens
with open('models/ebdt_data/p_model/evidence_run3.pl', 'w') as f:
    for s in evidence_dict:
        for p in evidence_dict[s]:
            for e in evidence_dict[s][p]:
                f.write(e + '\n')
            f.write('--------------------' + '\n')

LFI

In [9]:
DIR = 'models/ebdt_data/p_model/'
max_iter = 20
# learning from interpretation
cmd = f'problog lfi {DIR}p_model_lfi_run3.pl {DIR}evidence_run3.pl -O {DIR}p_model_run3_20i.pl --logger {DIR}log_run3-20i.txt -k ddnnf -v -n {max_iter}'
os.system(cmd)